##### my-zero-to-gbm-proj-assign

In [ ]:
import pandas as pd
import numpy as np
#import plotly.express as px
#train = pd.read_csv('../../../data/optiver-realized-volatility-prediction/train.csv')
train = pd.read_csv('d:\\optiver-realized-volatility-prediction\\train.csv')

In [ ]:
import glob
list_order_book_file_train = glob.glob('d:\\optiver-realized-volatility-prediction\\book_train.parquet/*')
list_trade_book_file_train = glob.glob('d:\\optiver-realized-volatility-prediction\\trade_train.parquet/*')
#list_order_book_file_train

In [ ]:
%who

In [ ]:
df_order_book = pd.DataFrame()
for file in list_order_book_file_train:
     df_stock_book = pd.read_parquet(file)
     df_stock_book['stock_id'] = file.split('=')[1]
     df_order_book = pd.concat([df_order_book,df_stock_book])


df_trade_book = pd.DataFrame()
for file in list_trade_book_file_train:
     df_stock_book = pd.read_parquet(file)
     df_stock_book['stock_id'] = file.split('=')[1]
     df_trade_book = pd.concat([df_trade_book,df_stock_book])


In [ ]:
# with open('book_index.csv','w+') as f:
#     for items in list(df_order_book_50.index):
#         f.write('%s\n' %items)
#new_index
# df_order_book_5.reset_index(inplace=True,drop=True)
# df_trade_book_5.reset_index(inplace=True,drop=True)

df_order_book.reset_index(inplace=True,drop=True)
df_trade_book.reset_index(inplace=True,drop=True)

#df_order_book_50.head(-1)
#df_order_book_50[df_order_book_50.index != df_order_book_50['index']]
#(72311913, 11)

In [ ]:
df_order_book['stock_id'] = df_order_book['stock_id'].astype('int8')
df_trade_book['stock_id'] = df_trade_book['stock_id'].astype('int8')

In [ ]:
df_order_book['order_wap'] = ((((df_order_book['bid_price1'].values * df_order_book['bid_size1'].values + df_order_book['bid_price2'].values * 
    df_order_book['bid_size2'].values)/(df_order_book['bid_size1'].values + df_order_book['bid_size2'].values)) * (df_order_book['ask_size1'].values + 
    df_order_book['ask_size2'].values)) + (((df_order_book['ask_price1'].values * df_order_book['ask_size1'].values + df_order_book['ask_price2'].values * 
    df_order_book['ask_size2'].values)/(df_order_book['ask_size1'].values + df_order_book['ask_size2'].values)) * (df_order_book['bid_size1'].values + 
    df_order_book['bid_size2'].values)) / ((df_order_book['bid_size1'].values + df_order_book['bid_size2'].values) +  (df_order_book['ask_size1'].values + 
    df_order_book['ask_size2'].values))).astype('float32')

In [ ]:
new_index = pd.Index(np.arange(0,600), name="seconds_in_bucket")

In [ ]:
def log_return(list_stock_prices):
    return np.log(list_stock_prices).diff()

In [ ]:

def order_volatility(series_log_return):
    return np.sqrt(np.sum(series_log_return**2))

In [ ]:
df_trade_book['scc'] = (df_trade_book['size']/df_trade_book['order_count']).values.astype('float16')

In [ ]:
reg = df_trade_book['price'].corr(df_trade_book['scc'])

In [ ]:
df_order_trade_merged = pd.merge(
    df_order_book[['time_id','stock_id','seconds_in_bucket','order_wap']],
    df_trade_book[['time_id','stock_id','seconds_in_bucket','price','scc']],
    how="outer",
    on=['time_id','stock_id','seconds_in_bucket'],
    sort=True,
    copy=False,
    indicator=False,
    validate='m:m'
    )

In [ ]:
del [df_order_book,df_trade_book]

In [ ]:
grouped = df_order_trade_merged.groupby(['time_id','stock_id'])['seconds_in_bucket','order_wap','price','scc']

In [ ]:
s1 = grouped.apply(lambda x : x.set_index('seconds_in_bucket').reindex(new_index))

In [ ]:
del grouped

In [ ]:
bf = s1.groupby(['time_id','stock_id'])

In [ ]:
s1.loc[s1.index.get_level_values(2)==0,['order_wap','price','scc']] = bf.first()

In [ ]:
del bf

In [ ]:
a = s1.loc(axis=0)[:,:,0][s1.loc(axis=0)[:,:,0]['price'].isna()][['price','scc']]

In [ ]:
def price_scc(col):
    #return  s1.loc(axis=0)[:,col.name[1],0][col.index[0]].mean(),s1.loc(axis=0)[:,col.name[1],:][col.index[1]].mean()
    #return s1.loc(axis=0)[:,col.name[1],0].groupby(level=[1,2])['price'].mean(), s1.loc(axis=0)[:,col.name[1],0].groupby(level=[1,2])['scc'].mean()
    return s1.loc(axis=0)[:,col.name[1],0]['price'].mean(), np.asscalar(s1.loc(axis=0)[:,col.name[1],0]['scc'].mean(skipna=True,level=[1,2]).values)
    #return col.name[1] , col.index[0]
    #row.scc = s1.loc(axis=0)[:,row.index.get_level_values(1),0]['scc'].mean()

In [ ]:
b = a.transform(price_scc ,axis=1)

In [ ]:
s1.at[b.index.values,['price','scc']] = b[['price','scc']].values

In [ ]:
s1.loc(axis=0)[:,:,0].isna().sum()

In [ ]:
s1['price'].describe()

In [ ]:
s2 = s1.groupby(['time_id','stock_id']).pad()

In [ ]:
s2.isna().sum()

In [ ]:
del [a,b,s1,df_order_trade_merged,df_stock_book]

Looping through each individual stocks, we can get the past realized volatility as prediction for each individual stocks.

In [ ]:
from sklearnex import patch_sklearn
patch_sklearn()

In [ ]:
from sklearn.preprocessing import Normalizer
#scaler = MinMaxScaler(feature_range=(-0.001,0.001))
scaler =  Normalizer()

In [ ]:
def scc_dev_fn(x):
    return scaler.fit_transform(x.values.reshape(-1,1))

In [ ]:
def trade_volatility(trade_wap_prices):
    return np.sqrt(np.sum(trade_wap_prices)/len(trade_wap_prices))


In [ ]:
s2['sccc'] = s2.groupby(['time_id','stock_id'])['scc'].transform(lambda x : x - np.mean(x.values)).astype('float16')

In [ ]:
s2[['price','sccc']].describe()

In [ ]:
s2['price_c'] = s2['price'] + pow(reg,2) * s2['sccc']

In [ ]:
s2['price_c'].describe()

In [ ]:
pd.options.display.float_format = '{:,.16f}'.format

In [ ]:
%who

In [ ]:
import sys
def sizeof_fmt(num, suffix='B'):
    ''' by Fred Cirera,  https://stackoverflow.com/a/1094933/1870254, modified'''
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f %s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f %s%s" % (num, 'Yi', suffix)

for name, size in sorted(((name, sys.getsizeof(value)) for name, value in locals().items()),
                         key= lambda x: -x[1])[:10]:
    print("{:>30}: {:>8}".format(name, sizeof_fmt(size)))

In [ ]:
s2.info()

In [ ]:
del s2['scc']
del s2['sccc']
del s2['price']

In [ ]:
s2['trade_wap'] = s2.groupby(['time_id','stock_id'])['price_c'].apply(lambda x: pow(x - x.mean(),2))

In [ ]:
import winsound
winsound.Beep(2500,3000)

In [ ]:
s2['trade_wap'].describe()

In [ ]:
s2['log_return'] = s2.groupby(['time_id','stock_id'])['order_wap'].apply(log_return)

In [ ]:
s2['log_return'].isna().sum()

In [ ]:
%who

In [ ]:
del [glob,list_order_book_file_train,list_trade_book_file_train,name,new_index,scaler,size,sizeof_fmt,price_scc,reg,]

In [ ]:
df_realized_vol_per_stock =  pd.DataFrame(s2[~s2['log_return'].isnull()]['log_return'].groupby(['time_id','stock_id']).agg(order_volatility))

In [ ]:
df_realized_vol_per_stock['row_id'] = df_realized_vol_per_stock.apply(lambda x: x.index.get_level_values(1).astype(str) + "-" + x.index.get_level_values(0).astype(str))

In [ ]:
df_realized_vol_per_stock['trade_return'] = pd.DataFrame(s2['trade_wap'].groupby(['time_id','stock_id']).agg(trade_volatility))

In [ ]:
winsound.Beep(2500,1000)

In [ ]:
df_realized_vol_per_stock.isna().sum()

In [ ]:
winsound.Beep(2500,3000)

In [ ]:
df_realized_vol_per_stock.info()

In [ ]:
train['row_id'] = train['stock_id'].astype(str) + '-' + train['time_id'].astype(str)
train = train[['row_id','target']]

In [ ]:
df_joined = train.merge(df_realized_vol_per_stock, on = ['row_id'], how = 'inner')

In [ ]:
df_joined.info()

In [ ]:
df_joined.isna().sum()

In [ ]:
inputs = df_joined[['log_return', 'trade_return']]
targets = df_joined['target']

In [ ]:
inputs.to_csv('inputs.csv')
targets.to_csv('targets.csv')

In [2]:
import pandas as pd
import numpy as np

In [3]:
inputs = pd.read_csv('inputs.csv',index_col=0)
targets = pd.read_csv('targets.csv',index_col=0)

In [4]:
inputs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 428932 entries, 0 to 428931
Data columns (total 2 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   log_return    428932 non-null  float64
 1   trade_return  428932 non-null  float64
dtypes: float64(2)
memory usage: 9.8 MB


In [5]:
targets.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 428932 entries, 0 to 428931
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   target  428932 non-null  float64
dtypes: float64(1)
memory usage: 6.5 MB


In [6]:
from sklearnex import patch_sklearn
patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [7]:
from sklearn.metrics import r2_score
# def rmspe(y_true, y_pred):
#     return  (np.sqrt(np.mean(np.square((y_true - y_pred) / y_true)))) * 100

In [8]:
from sklearn.model_selection import ShuffleSplit, RepeatedKFold

In [9]:
# ss = ShuffleSplit(n_splits = 5, test_size = 0.25, random_state=11111)
ss = RepeatedKFold(n_splits=2, n_repeats=3, random_state=11111)

In [10]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(3)
poly_inputs = pd.DataFrame(poly.fit_transform(inputs)).astype('float32')

In [11]:
from sklearn.metrics import mean_squared_error

In [12]:
from lightgbm import LGBMRegressor

In [13]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

In [14]:
# explicitly require this experimental feature
from sklearn.experimental import enable_halving_search_cv # noqa
# now you can import normally from model_selection
from sklearn.model_selection import HalvingRandomSearchCV
from sklearn.model_selection import HalvingGridSearchCV

In [15]:
import sklearn

In [16]:
sklearn. __version__

'1.0'

In [30]:
tuple(targets)

('target',)

In [69]:
def train_and_evaluate(X_train, train_targets, X_val, val_targets, **params):
    model = modelc
    model.fit(X_train, train_targets)#,early_stopping_rounds=5,eval_set=(X_val,val_targets))
    pred_train = model.predict(X_train)
    pred_val = model.predict(X_val)
    train_R2 = round(r2_score(train_targets,pred_train ),3)
    train_RMSPE = round(mean_squared_error(train_targets, pred_train,squared=False),3)
    val_R2 = round(r2_score(val_targets, pred_val),3)
    val_RMSPE = round(mean_squared_error(val_targets, pred_val,squared=False),3)

    # train_rmse = rmse(model.predict(X_train), train_targets)
    # val_rmse = rmse(model.predict(X_val), val_targets)
    return model, train_R2, train_RMSPE, val_R2, val_RMSPE
#n_jobs = -1 means that use all the available threads in that machine where the alogorithm is running 

In [18]:
# models = []
# for train_idxs, val_idxs in ss.split(inputs):
#     X_train, train_targets = inputs.iloc[train_idxs], targets.iloc[train_idxs]
#     X_val, val_targets = inputs.iloc[val_idxs], targets.iloc[val_idxs]
#     X_train = np.ascontiguousarray(X_train).reshape(-1,2)
#     train_targets = np.ascontiguousarray(train_targets).reshape(-1,1)
#     X_val = np.ascontiguousarray(X_val).reshape(-1,2)
#     val_targets = np.ascontiguousarray(val_targets).reshape(-1,1)
#     reg = LazyRegressor(ignore_warnings=False, random_state=11111, verbose=True)
#     models, predictions = reg.fit(X_train, X_val, train_targets, val_targets)  # pass all sets
#     print(models.head(100))

In [19]:
# models = []

# for train_idxs, val_idxs in ss.split(inputs):
#    X_train, train_targets = inputs.iloc[train_idxs], targets.iloc[train_idxs]
#    X_val, val_targets = inputs.iloc[val_idxs], targets.iloc[val_idxs]
#    X_train = np.ascontiguousarray(X_train).reshape(-1,2)
#    train_targets = np.ascontiguousarray(train_targets).reshape(-1,1)
#    X_val = np.ascontiguousarray(X_val).reshape(-1,2)
#    val_targets = np.ascontiguousarray(val_targets).reshape(-1,1)
#    model, train_R2, train_RMSPE, val_R2, val_RMSPE = train_and_evaluate(X_train, 
#                                                    train_targets, 
#                                                    X_val, 
#                                                    val_targets, 
#                                                    n_estimators=1000,
#                                                    booster='gbtree',
#                                                    verbosity=2)
#    models.append(model)
#    print('Train R2: {}, Train RMSPE: {}, Validation R2: {}, Validation RMSPE: {}'.format(train_R2, train_RMSPE, val_R2, val_RMSPE))

In [20]:
# model = XGBRegressor(random_state=11111, n_jobs=-1,verbosity=1,objective='reg:tweedie')
# param_grid = {"n_estimators" : [50,100,500],
#               "learning_rate": [0.1,0.01,0.001],
#               "booster": ['gbtree', 'gblinear'],
#               # "num_parallel_tree": range(1,6,1),
#               # "reg_alpha": np.logspace(-3,3,300),
#               # "reg_lambda": np.logspace(-3,3,300)
#             }

In [21]:
# model = HistGradientBoostingRegressor(random_state=11111,verbose=1)
# param_grid = {"max_leaf_nodes" : [15,31,63],
#               "learning_rate": [0.1,0.01,0.001],
#               "max_iter": [100,300],
#               "min_samples_leaf": [10,20,30],
#               "l2_regularization": [0.0,0.1,0.01]
#              }

In [22]:
# models = []

# for train_idxs, val_idxs in ss.split(inputs):
#    X_train, train_targets = inputs.iloc[train_idxs], targets.iloc[train_idxs]
#    X_val, val_targets = inputs.iloc[val_idxs], targets.iloc[val_idxs]
#    X_train = np.ascontiguousarray(X_train).reshape(-1,2)
#    train_targets = np.ascontiguousarray(train_targets).ravel()
#    X_val = np.ascontiguousarray(X_val).reshape(-1,2)
#    val_targets = np.ascontiguousarray(val_targets).ravel()
#    grid_search = HalvingGridSearchCV(model, param_grid,factor=2,random_state=11111,n_jobs=-1,verbose=1).fit(X_train, train_targets)
#    model, train_R2, train_RMSPE, val_R2, val_RMSPE = train_and_evaluate(X_train, 
#                                                    train_targets, 
#                                                    X_val, 
#                                                    val_targets, 
#                                                    **grid_search.best_params_)
#    models.append(model)
#    print('Train R2: {}, Train RMSPE: {}, Validation R2: {}, Validation RMSPE: {}'.format(train_R2, train_RMSPE, val_R2, val_RMSPE))

In [23]:
# modelc = RandomForestRegressor(random_state=11111,verbose=1,n_jobs=-1)
# param_grid = {"n_estimators": [50,100,500],
#               "min_samples_split": [2,4,6],
#               "min_samples_leaf": [1,2,4],
#             #   "min_impurity_decrease": [0.0,0.01,0.001],
#             #   "oob_score": [True,False],
#             #   "ccp_alpha": [0.0,0.01,0.001],
#              }

In [62]:
params = {'num_leaves':31}
modelc = LGBMRegressor(random_state=11111,n_jobs=-1,silent=False,metric='rmse',force_col_wise=True,**params)#,bagging_freq= 5, bagging_fraction= 0.75)
param_grid = {"boosting_type": ['gbdt','goss','dart'],
              "learning_rate" : [0.1,0.01,0.001],
              "n_estimators": [50,100,300],
              "min_child_samples": [10,20,30],
              "min_split_gain": [0.0,0.001,0.1],
            }

In [71]:
X_train.shape

(214466, 10)

In [70]:
#without early stopping
params = grid_search.best_params_
model, train_R2, train_RMSPE, val_R2, val_RMSPE = train_and_evaluate(X_train, 
                                                train_targets, 
                                                X_val, 
                                                val_targets, 
                                                **params)

[LightGBM] [Info] Total Bins 2295
[LightGBM] [Info] Number of data points in the train set: 214466, number of used features: 9
[LightGBM] [Info] Start training from score 0.003882


In [63]:
#with early stopping
params = grid_search.best_params_
model, train_R2, train_RMSPE, val_R2, val_RMSPE = train_and_evaluate(X_train, 
                                                train_targets, 
                                                X_val, 
                                                val_targets, 
                                                **params)

[LightGBM] [Info] Total Bins 2295
[LightGBM] [Info] Number of data points in the train set: 214466, number of used features: 9
[LightGBM] [Info] Start training from score 0.003882
[1]	valid_0's rmse: 0.00279718
Training until validation scores don't improve for 5 rounds
[2]	valid_0's rmse: 0.00268065
[3]	valid_0's rmse: 0.00258229
[4]	valid_0's rmse: 0.0024995
[5]	valid_0's rmse: 0.00243036
[6]	valid_0's rmse: 0.00237282
[7]	valid_0's rmse: 0.00232505
[8]	valid_0's rmse: 0.00228531
[9]	valid_0's rmse: 0.00225271
[10]	valid_0's rmse: 0.00222552
[11]	valid_0's rmse: 0.00220341
[12]	valid_0's rmse: 0.00218524
[13]	valid_0's rmse: 0.00217036
[14]	valid_0's rmse: 0.00215838
[15]	valid_0's rmse: 0.00214826
[16]	valid_0's rmse: 0.00214009
[17]	valid_0's rmse: 0.00213362
[18]	valid_0's rmse: 0.00212821
[19]	valid_0's rmse: 0.00212385
[20]	valid_0's rmse: 0.00212023
[21]	valid_0's rmse: 0.00211729
[22]	valid_0's rmse: 0.00211505
[23]	valid_0's rmse: 0.0021131
[24]	valid_0's rmse: 0.0021116
[25]

In [56]:
models = []
model_params = []
grid_params = []
for train_idxs, val_idxs in ss.split(poly_inputs):
   X_train, train_targets = poly_inputs.iloc[train_idxs], targets.iloc[train_idxs]
   X_val, val_targets = poly_inputs.iloc[val_idxs], targets.iloc[val_idxs]
   X_train = np.ascontiguousarray(X_train).reshape(-1,10)
   train_targets = np.ascontiguousarray(train_targets).ravel()
   X_val = np.ascontiguousarray(X_val).reshape(-1,10)
   val_targets = np.ascontiguousarray(val_targets).ravel()
   grid_search = HalvingGridSearchCV(modelc, param_grid,n_jobs=-1,verbose=1,return_train_score=True,random_state=11111,factor=3,min_resources=1000,error_score='raise').fit(X_train, train_targets)
   params = grid_search.best_params_
   model, train_R2, train_RMSPE, val_R2, val_RMSPE = train_and_evaluate(X_train, 
                                                   train_targets, 
                                                   X_val, 
                                                   val_targets, 
                                                   **params)
   models.append(model)
   model_params.append(model.get_params())
   grid_params.append(grid_search.best_params_)
   print('Train R2: {}, Train RMSPE: {}, Validation R2: {}, Validation RMSPE: {}'.format(train_R2, train_RMSPE, val_R2, val_RMSPE))

n_iterations: 5
n_required_iterations: 5
n_possible_iterations: 5
min_resources_: 1000
max_resources_: 214466
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 243
n_resources: 1000
Fitting 5 folds for each of 243 candidates, totalling 1215 fits


KeyboardInterrupt: 

In [ ]:
import lightgbm as lgb

In [ ]:
train_data = lgb.Dataset(poly_inputs.values,label=targets.values,free_raw_data=True)
validation_data = lgb.Dataset('validation.svm', reference=train_data,free_raw_data=True)

In [ ]:
lgb.train()

In [48]:
models

[LGBMRegressor(metric='rmse', random_state=11111, silent=False),
 LGBMRegressor(metric='rmse', random_state=11111, silent=False),
 LGBMRegressor(metric='rmse', random_state=11111, silent=False),
 LGBMRegressor(metric='rmse', random_state=11111, silent=False),
 LGBMRegressor(metric='rmse', random_state=11111, silent=False),
 LGBMRegressor(metric='rmse', random_state=11111, silent=False)]

In [25]:
print(modelc.get_params()) #HalvingGridSearch

{'boosting_type': 'gbdt', 'class_weight': None, 'colsample_bytree': 1.0, 'importance_type': 'split', 'learning_rate': 0.1, 'max_depth': -1, 'min_child_samples': 20, 'min_child_weight': 0.001, 'min_split_gain': 0.0, 'n_estimators': 100, 'n_jobs': -1, 'num_leaves': 31, 'objective': None, 'random_state': 11111, 'reg_alpha': 0.0, 'reg_lambda': 0.0, 'silent': False, 'subsample': 1.0, 'subsample_for_bin': 200000, 'subsample_freq': 0, 'metric': 'rmse'}


In [42]:
modelc.feature_importances_

array([  0, 449, 286,   0, 165,   0,   0, 278, 232,   0])

In [43]:
modelc.objective_

'regression'

In [45]:
df_gd_par = pd.DataFrame(grid_params)
df_md_par = pd.DataFrame(model_params)

In [46]:
df_gd_par

,boosting_type,learning_rate,min_child_samples,min_split_gain,n_estimators
0,goss,0.01,20,0.0,300
1,goss,0.01,20,0.0,300
2,goss,0.01,20,0.0,300
3,goss,0.01,10,0.0,300
4,goss,0.01,10,0.0,300
5,goss,0.01,10,0.0,300


In [47]:
df_md_par

,boosting_type,class_weight,colsample_bytree,importance_type,learning_rate,max_depth,min_child_samples,min_child_weight,min_split_gain,n_estimators,...,num_leaves,objective,random_state,reg_alpha,reg_lambda,silent,subsample,subsample_for_bin,subsample_freq,metric
0,gbdt,None,1.0,split,0.1,-1,20,0.001,0.0,100,...,31,None,11111,0.0,0.0,False,1.0,200000,0,rmse
1,gbdt,None,1.0,split,0.1,-1,20,0.001,0.0,100,...,31,None,11111,0.0,0.0,False,1.0,200000,0,rmse
2,gbdt,None,1.0,split,0.1,-1,20,0.001,0.0,100,...,31,None,11111,0.0,0.0,False,1.0,200000,0,rmse
3,gbdt,None,1.0,split,0.1,-1,20,0.001,0.0,100,...,31,None,11111,0.0,0.0,False,1.0,200000,0,rmse
4,gbdt,None,1.0,split,0.1,-1,20,0.001,0.0,100,...,31,None,11111,0.0,0.0,False,1.0,200000,0,rmse
5,gbdt,None,1.0,split,0.1,-1,20,0.001,0.0,100,...,31,None,11111,0.0,0.0,False,1.0,200000,0,rmse


In [27]:
models = []

for train_idxs, val_idxs in ss.split(poly_inputs):
   X_train, train_targets = poly_inputs.iloc[train_idxs], targets.iloc[train_idxs]
   X_val, val_targets = poly_inputs.iloc[val_idxs], targets.iloc[val_idxs]
   X_train = np.ascontiguousarray(X_train).reshape(-1,10)
   train_targets = np.ascontiguousarray(train_targets).ravel()
   X_val = np.ascontiguousarray(X_val).reshape(-1,10)
   val_targets = np.ascontiguousarray(val_targets).ravel()
   grid_search = GridSearchCV(modelc, param_grid,n_jobs=-1,verbose=1,return_train_score=True,error_score='raise').fit(X_train, train_targets)
   model, train_R2, train_RMSPE, val_R2, val_RMSPE = train_and_evaluate(X_train, 
                                                   train_targets, 
                                                   X_val, 
                                                   val_targets, 
                                                   **grid_search.best_params_)
   models.append(modelc)
   print('Train R2: {}, Train RMSPE: {}, Validation R2: {}, Validation RMSPE: {}'.format(train_R2, train_RMSPE, val_R2, val_RMSPE))

Fitting 5 folds for each of 243 candidates, totalling 1215 fits


KeyboardInterrupt: 